In [1]:
from exotic_pricing_HX import ExoticPricing
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np

%matplotlib inline

In [2]:
t = datetime.timestamp(datetime.strptime('20210603-00:00:00', "%Y%m%d-%H:%M:%S"))
T = datetime.timestamp(datetime.strptime('20220603-00:00:00', "%Y%m%d-%H:%M:%S"))
T = (T - t) / 60 / 60 / 24 / 365

# initialize parameters
S0 = 35  # e.g. spot price = 35
K = 40  # e.g. exercise price = 40
T = T  # e.g. one year
r = 0.01  # e.g. risk free rate = 1%
sigma = 0.5  # e.g. volatility = 5%
npath = 10000  # no. of slices PER YEAR e.g. quarterly adjusted or 252 trading days adjusted

# optional parameter
simulation_rounds = int(100)  # For monte carlo simulation, a large number of simulations required

MC = ExoticPricing(S0=S0,
                       K=K,
                       T=T,
                       r=r,
                       sigma=sigma,
                       simulation_rounds=simulation_rounds,
                       npath=npath,
                       fix_random_seed=2)

MC.stock_price_simulation()

----------------------------------------------------------------
 Number of simulations  100 
 S0 35.0 
 K 40.0 
 Maximum Stock price 267.39 
 Minimum Stock price 4.24 
 Average stock price 35.684 
 Standard Error 0.19213 
----------------------------------------------------------------


(35.68417639235075, 0.1921251981957408)

In [3]:
def local_vol(dt,dk,S0,K,T,r,sigma,simulation_rounds,npath):

    # 计算每个时点的期权的价格
    barrier_price = S0 * 1.3
    parisian_barrier_days = 21

    MC1 = ExoticPricing(S0=S0,
                           K=K,
                           T=T,
                           r=r,
                           sigma=sigma,
                           simulation_rounds=simulation_rounds,
                           npath=npath,
                           fix_random_seed=2)
    # MC1.CIR_model(a=0.5, b=0.05, sigma_r=0.1)
    # MC1.heston(kappa=2, theta=0.3, sigma_v=0.3, rho=0.5)
    MC1.stock_price_simulation()

    C0 = MC1.barrier_option(option_type="call",
                               barrier_price=barrier_price,
                               barrier_type="knock-in",
                               barrier_direction="down",
                               parisian_barrier_days=parisian_barrier_days)[0]

    # 计算此时点上dc by dt
    dt = dt
    dk = dk

    MC1 = ExoticPricing(S0=S0,
                           K=K,
                           T=T + dt,
                           r=r,
                           sigma=sigma,
                           simulation_rounds=simulation_rounds,
                           npath=npath,
                           fix_random_seed=2)
    # MC1.CIR_model(a=0.5, b=0.05, sigma_r=0.1)
    # MC1.heston(kappa=2, theta=0.3, sigma_v=0.3, rho=0.5)
    MC1.stock_price_simulation()
    c_t = MC1.barrier_option(option_type="call",
                                barrier_price=barrier_price,
                                barrier_type="knock-in",
                                barrier_direction="down",
                                parisian_barrier_days=parisian_barrier_days)[0]

    dc_dt = (c_t - C0)/dt

    # dc by dk
    K1 = K + dk  # e.g. exercise price = 40

    MC1 = ExoticPricing(S0=S0,
                           K=K1,
                           T=T,
                           r=r,
                           sigma=sigma,
                           simulation_rounds=simulation_rounds,
                           npath=npath,
                           fix_random_seed=2)
    # MC1.CIR_model(a=0.5, b=0.05, sigma_r=0.1)
    # MC1.heston(kappa=2, theta=0.3, sigma_v=0.3, rho=0.5)
    MC1.stock_price_simulation()
    c_k1 = MC1.barrier_option(option_type="call",
                                 barrier_price=barrier_price,
                                 barrier_type="knock-in",
                                 barrier_direction="down",
                                 parisian_barrier_days=parisian_barrier_days)[0]
    dc_dk1 = (c_k1 - C0) / dk

    K2 = K - dk  # e.g. exercise price = 40
    T = T  # e.g. one year

    MC1 = ExoticPricing(S0=S0,
                           K=K2,
                           T=T,
                           r=r,
                           sigma=sigma,
                           simulation_rounds=simulation_rounds,
                           npath=npath,
                           fix_random_seed=501)
    # MC1.CIR_model(a=0.5, b=0.05, sigma_r=0.1)
    # MC1.heston(kappa=2, theta=0.3, sigma_v=0.3, rho=0.5)
    MC1.stock_price_simulation()
    c_k2 = MC1.barrier_option(option_type="call",
                                 barrier_price=barrier_price,
                                 barrier_type="knock-in",
                                 barrier_direction="down",
                                 parisian_barrier_days=parisian_barrier_days)[0]
    d2c_dk2 = (c_k1 - 2 * C0 + c_k2) / dk ** 2

    local_vol = np.sqrt((dc_dt + r * C0 - (r - 0) * (
                C0 - K * dc_dk1)) / (0.5 * K * K * d2c_dk2))
    return np.array([C0,local_vol])


In [4]:
true_path  = MC.price_array[:,np.random.randint(npath)]
n = 0
dt = 1/252
dk = 0.5
local_V = np.zeros([2,simulation_rounds])
for i in range(len(true_path)):
    S = true_path[i]
    T = T - n * (T/npath)
    local_V[:,i]=local_vol(dt, dk, S, K, T, r, sigma, simulation_rounds, npath)
    n += 1

# local_V[1, np.where(np.isnan(local_V))[1]] = np.interp(local_V[0, np.where(np.isnan(local_V))[1]],local_V[0],local_V[1])

----------------------------------------------------------------
 Number of simulations  100 
 S0 35.0 
 K 40.0 
 Maximum Stock price 216.21 
 Minimum Stock price 5.50 
 Average stock price 35.409 
 Standard Error 0.18553 
----------------------------------------------------------------


NameError: name 'MC' is not defined

In [5]:
local_vol(dt, dk, S, K, T, r, sigma, simulation_rounds, npath)

----------------------------------------------------------------
 Number of simulations  100 
 S0 35.0 
 K 40.0 
 Maximum Stock price 207.52 
 Minimum Stock price 4.98 
 Average stock price 35.204 
 Standard Error 0.18600 
----------------------------------------------------------------


NameError: name 'MC' is not defined

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.subplot(211)
plt.plot(true_path,label='stock price')

plt.subplot(212)
plt.plot(local_V[0],color = 'r',label = 'barrier opt price')
plt.plot(local_V[1],color = 'orange',label = 'option local vol')

lines = []
labels = []
for ax in fig.axes:
    axLine, axLabel = ax.get_legend_handles_labels()
    lines.extend(axLine)
    labels.extend(axLabel)

fig.legend(lines, labels,
           loc = 'upper right')

plt.show()

In [7]:
# 计算每个时点的期权的价格
barrier_price = S0 * 1.3
parisian_barrier_days = 21

MC = ExoticPricing(S0=S0,
                           K=K,
                           T=T,
                           r=r,
                           sigma=sigma,
                           simulation_rounds=simulation_rounds,
                           npath=npath,
                           fix_random_seed=2)
    # MC.CIR_model(a=0.5, b=0.05, sigma_r=0.1)
    # MC.heston(kappa=2, theta=0.3, sigma_v=0.3, rho=0.5)
MC.stock_price_simulation()

C0 = MC.barrier_option('call',)

----------------------------------------------------------------
 Number of simulations  100 
 S0 35.0 
 K 40.0 
 Maximum Stock price 207.52 
 Minimum Stock price 4.98 
 Average stock price 35.204 
 Standard Error 0.18600 
----------------------------------------------------------------


NameError: name 'MC' is not defined